# Coleta de dados da busca em largura (BFS)

In [ ]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import time
import tracemalloc
from collections import deque
import pandas as pd
import sys

def bfs_resolver_labirinto_com_visualizacao(labirinto, inicio, fim):
    altura, largura = len(labirinto), len(labirinto[0])
    visitado = [[False for _ in range(largura)] for _ in range(altura)]
    pai = [[None for _ in range(largura)] for _ in range(altura)]
    fila = deque()
    nos_visitados = set()

    fila.append(inicio)
    visitado[inicio[1]][inicio[0]] = True

    tracemalloc.start()
    tempo_inicio = time.perf_counter()

    while fila:
        x, y = fila.popleft()
        nos_visitados.add((x, y))

        if (x, y) == fim:
            break

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < largura and 0 <= ny < altura:
                if labirinto[ny][nx] == 1 and not visitado[ny][nx]:
                    fila.append((nx, ny))
                    visitado[ny][nx] = True
                    pai[ny][nx] = (x, y)

    tempo_fim = time.perf_counter()
    memoria_usada = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    caminho = []
    x, y = fim
    while (x, y) != inicio and pai[y][x] is not None:
        caminho.append((x, y))
        x, y = pai[y][x]
    caminho.append(inicio)
    caminho.reverse()

    return {
        "caminho": caminho,
        "nos_visitados": list(nos_visitados),
        "tempo_ms": (tempo_fim - tempo_inicio) * 1000,
        "memoria_bytes": memoria_usada,
        "comprimento": len(caminho),
        "qtd_nos_visitados": len(nos_visitados)
    }

## Resolução de lote de labirintos

In [ ]:
def executar_bfs_em_lote(caminho_labirintos, tamanho_nome, output_csv):
    resultados = []

    for i in range(1, 21):
        lab_path = os.path.join(caminho_labirintos, f"labirinto_{i:02d}.npy")
        if not os.path.exists(lab_path):
            print(f"Labirinto não encontrado: {lab_path}")
            continue

        labirinto = np.load(lab_path)
        entrada = (0, 1)
        saida = (labirinto.shape[1] - 1, labirinto.shape[0] - 2)

        resultado = bfs_resolver_labirinto_com_visualizacao(labirinto, entrada, saida)
        print(f"Labirinto {i}/20")
        resultados.append([
            "DFS",
            tamanho_nome,
            i,
            round(resultado["tempo_ms"], 2),
            resultado["qtd_nos_visitados"],
            bool(resultado["comprimento"] > 0),
            resultado["comprimento"],
            round(resultado["memoria_bytes"] / 1024, 2)  # em KB
        ])

    # Salva CSV
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "algoritmo", "tamanho_labirinto", "indice_labirinto",
            "tempo_execucao_ms", "nos_visitados", "encontrou_caminho",
            "comprimento_caminho", "memoria_kb"
        ])
        writer.writerows(resultados)

    print(f"✅ Resultados salvos em: {output_csv}")


## Resolução do lote com labirintos pequenos

In [ ]:
executar_dfs_em_lote("labirintos/pequenos", "pequeno", "resultados/bfs/bfs_pequenos.csv")


## Resolução do lote com labirintos médios

In [ ]:
executar_dfs_em_lote("labirintos/medios", "medio", "resultados/bfs/bfs_medios.csv")


## Resolução do lote com labirintos médios-grandes

In [ ]:
executar_dfs_em_lote("labirintos/medio-grande", "medio-grande", "resultados/bfs/bfs_medios_grandes.csv")


## Resolução do lote com labirintos grandes

In [ ]:
executar_dfs_em_lote("labirintos/grandes", "grande", "resultados/bfs/bfs_grandes.csv")
